# Fine-tuned Based Models for sentiment analysis

In [1]:
import joblib
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertConfig


from sklearn.metrics import classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt

/home/luisdogo/miniconda3/envs/sentiment-deep_learning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-06 01:41:10.875742: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-06 01:41:10.953439: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Read Embeddings & labels

In [2]:
X_train = joblib.load('data/train_embeddings.pkl')
y_train = pd.read_csv('data/training.csv')['label'].values

X_val = joblib.load('data/validation_embeddings.pkl')
y_val = pd.read_csv('data/validation.csv')['label'].values

X_test = joblib.load('data/test_embeddings.pkl')
y_test = pd.read_csv('data/test.csv')['label'].values

In [3]:
print(X_train.shape, y_train.shape)
print(type(X_train), type(y_train))

(16000, 768) (16000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


## Transform Dataframe into Dataloader

In [4]:
class EmotionDataset(Dataset): # defines how the text is pre-processed before sending it to the neural network.
    # generates 
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings  # Lista de embeddings generados por RoBERTa
        self.labels = labels.squeeze()  # Lista de etiquetas

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): # []
        return torch.tensor(self.embeddings[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [5]:
# Dataset type conversion
train_dataset = EmotionDataset(X_train, y_train)
val_dataset = EmotionDataset(X_val, y_val)
test_dataset = EmotionDataset(X_test, y_test)

# will feed the data in batches to the neural network for suitable training and processing.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(type(train_dataset), type(train_loader))

<class '__main__.EmotionDataset'> <class 'torch.utils.data.dataloader.DataLoader'>


# BERT model

Parameters:
- Input_size: Embedding length
- Heads: Number of heads

In [32]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)  # Classifier layer in top of the model

    def forward(self, embeddings):
        output_2 = self.l2(embeddings)
        output = self.l3(output_2)
        return output

In [7]:
device = 'cuda' if cuda.is_available() else 'cpu'
model = BERTClass()
model.to(device)

### Training

In [10]:
# hyperparameters
LEARNING_RATE = .01
EPOCHS = 10
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [9]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

In [31]:
def train(epoch):
    loss_history = []
    model.train()
    for _, data in enumerate(train_loader, 0):
        # Unpack the data from the DataLoader
        embeddings, targets = data
        embeddings = embeddings.to(device, dtype=torch.float32)  # Move embeddings to the device
        targets = targets.to(device, dtype=torch.long)  # Move targets to the device

        # Forward pass through the model
        outputs = model(embeddings)

        # Calculate loss
        optimizer.zero_grad()
        # One-hot encoding
        one_hot = torch.nn.functional.one_hot(targets, num_classes=6).float() # loss function needs it like this, idk
        loss = loss_fn(outputs, one_hot)
        loss_history.append(loss.detach())
        if _ % 5000 == 0:  # Print loss for every 5000 steps
            print(f'Epoch: {epoch}, Loss: {loss.item()}')
        # Backward pass and optimization step
        loss.backward()
        optimizer.step()
    return loss_history

In [26]:
for epoch in range(EPOCHS):
    l_h = train(epoch)

Epoch: 0, Loss: 1.7629598379135132
Epoch: 1, Loss: 1.165524959564209
Epoch: 2, Loss: 0.9943251013755798
Epoch: 3, Loss: 1.0869170427322388
Epoch: 4, Loss: 1.3888359069824219
Epoch: 5, Loss: 1.1776103973388672
Epoch: 6, Loss: 1.442686676979065
Epoch: 7, Loss: 0.787227988243103
Epoch: 8, Loss: 1.5459790229797363
Epoch: 9, Loss: 1.3632638454437256
Epoch: 10, Loss: 1.7111494541168213
Epoch: 11, Loss: 0.8610047101974487
Epoch: 12, Loss: 0.7785718441009521
Epoch: 13, Loss: 1.6410508155822754
Epoch: 14, Loss: 1.010100245475769
Epoch: 15, Loss: 1.3328192234039307
Epoch: 16, Loss: 1.7329339981079102
Epoch: 17, Loss: 1.5961849689483643
Epoch: 18, Loss: 1.3418307304382324
Epoch: 19, Loss: 1.6282483339309692
Epoch: 20, Loss: 1.4765093326568604
Epoch: 21, Loss: 0.8197331428527832
Epoch: 22, Loss: 1.3600597381591797
Epoch: 23, Loss: 1.3956938982009888
Epoch: 24, Loss: 1.3879404067993164
Epoch: 25, Loss: 1.1671992540359497
Epoch: 26, Loss: 1.7250717878341675
Epoch: 27, Loss: 1.6508972644805908
Epoch:

### Evaluation

In [14]:
def test(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
            embeddings, targets = data
            embeddings = embeddings.to(device, dtype=torch.float32)  # Move embeddings to the device
            targets = targets.to(device, dtype=torch.long)  # Move targets to the device
            outputs = model(embeddings)
            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [15]:
for epoch in range(EPOCHS):
    outputs, targets = test(epoch)
    results = np.array([np.argmax(inner) for inner in outputs])
    accuracy = accuracy_score(targets, results)
    f1_score_micro = f1_score(targets, results, average='micro')
    f1_score_macro = f1_score(targets, results, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score (Micro) = 0.6125
F1 Score (Macro) = 0.5329566168215091
Accuracy Score = 0.6125
F1 Score

In [16]:
# pick base pre-trained model

# define feature extraction layer(docking layer to classification head), usually "bottle-neck layer"(last layer before flatten op)

#Feature extraction
#In this step, you will freeze the convolutional base created from the previous step and to use as a feature extractor
# #Additionally, you add a classifier on top of it and train the top-level classifier.


In [17]:
# define the classification head (class)



### Hyperparameters

In [ ]:
values = [tensor.item() for tensor in tensor_list]

# Plot the values
plt.plot(values, marker='o')
plt.title("Graph of Extracted Values")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid()
plt.show()

### Visualization

In [18]:
# training & Validation Loss
# Training & Validation F1-score
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


NameError: name 'history' is not defined

## RoBERTa model

In [ ]:
# will be done if we finish the bert model on time